In [55]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import chromadb

In [2]:
load_dotenv()

True

In [3]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [4]:
llm = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
    api_key=GROQ_API_KEY
)

In [7]:
response = llm.invoke(input="Create a python code to sum the numbers some two numbers")

In [8]:
print(response.content)

Here is a simple Python code to sum two numbers:

```python
# Function to add two numbers
def add_two_numbers(num1, num2):
  return num1 + num2

# Take input from user
num1 = float(input("Enter first number: "))
num2 = float(input("Enter second number: "))

# Call the function to add two numbers
sum = add_two_numbers(num1, num2)

# Print the sum
print("The sum of {0} and {1} is {2}".format(num1, num2, sum))
```

In this code, we define a function `add_two_numbers` that takes two arguments and returns their sum. We then take input from the user for two numbers, call the function to add them, and print the result.


In [9]:
chroma_client = chromadb.Client()

In [10]:
collection = chroma_client.get_or_create_collection(
    name="my_collection"
)

In [11]:
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2" ]
)

In [12]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This is a document about pineapple',
  'This is a document about oranges'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [13]:
results = collection.query(query_texts="This is the query document about Assam",
                           n_results=2)

In [14]:
results

{'ids': [['id1', 'id2']],
 'distances': [[1.4745726585388184, 1.6186896562576294]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This is a document about pineapple',
   'This is a document about oranges']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [74]:
loader = WebBaseLoader("https://jobs.nike.com/job/R-39879")

In [75]:
page_data = loader.load().pop().page_content

In [76]:
print(page_data)

Apply for Machine Learning Engineer - Supply Chain

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsMachine Learning Engineer - Supply ChainBeaverton, OregonBecome a Part

In [77]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the vaild JSON.
    ### VAILD JSON (NO PREAMBLE):
    """
)

In [78]:
chain_extract = prompt_extract | llm

In [79]:
res = chain_extract.invoke(input={'page_data': page_data})

In [80]:
print(res.content)

[
    {
        "role": "Machine Learning Engineer - Supply Chain",
        "experience": "1-3 years of professional experience in software engineering, data engineering, machine learning, or related field",
        "skills": [
            "Python",
            "Docker",
            "CI/CD",
            "agile development",
            "test driven development",
            "data structures",
            "data modeling",
            "software architecture",
            "data engineering",
            "ETL pipelines",
            "SQL",
            "MLOps",
            "API development",
            "mathematical optimization",
            "Amazon Web Services",
            "Spark",
            "Kubernetes",
            "Jenkins",
            "Databricks",
            "Terraform"
        ],
        "description": "As a Machine Learning Engineer, you will develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. You sho

In [81]:
type(res.content)

str

In [82]:
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)[0]
json_res

{'role': 'Machine Learning Engineer - Supply Chain',
 'experience': '1-3 years of professional experience in software engineering, data engineering, machine learning, or related field',
 'skills': ['Python',
  'Docker',
  'CI/CD',
  'agile development',
  'test driven development',
  'data structures',
  'data modeling',
  'software architecture',
  'data engineering',
  'ETL pipelines',
  'SQL',
  'MLOps',
  'API development',
  'mathematical optimization',
  'Amazon Web Services',
  'Spark',
  'Kubernetes',
  'Jenkins',
  'Databricks',
  'Terraform'],
 'description': 'As a Machine Learning Engineer, you will develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. You should have experience in Python, a strong background in algorithms and data structures, hands-on AWS experience, as well as experience in database technology (e.g. Postgres, Redis) and data processing technology (e.g. EMR). You should also have a dem

In [83]:
type(json_res)

dict

In [102]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are praveen, a fresher actively seeking opportunities in the fields of Artificial Intelligence (AI), Machine Learning (ML), and Generative AI (GenAI). I have a solid foundation in the following skills and technologies:

    Programming: Python, SQL
    Machine Learning & Deep Learning: Supervised/Unsupervised Learning, Neural Networks
    Natural Language Processing (NLP): Text processing, transformers, language models
    Large Language Models (LLM): Working with GPT-like models, fine-tuning, prompt engineering
    MLOps & LLOps: End-to-end ML/LLM pipeline, model deployment, monitoring
    Cloud Services: AWS (ECR, EC2)
    Version Control: GitHub (CI/CD pipelines)
    
    I have successfully completed multiple projects in the domains of machine learning, deep learning, 
    NLP, LLM, and MLOps. I'm familiar with the latest industry trends and have implemented solutions in 
    real-world projects, from data preprocessing to model deployment on cloud platforms.

    My objective is to explore entry-level roles or internships in AI/ML/GenAI that leverage my skills 
    and offer opportunities for growth. I am particularly interested in positions that allow me to work 
    on cutting-edge technologies and contribute to impactful solutions.
    
    Your job is to write a cold email to the client regarding the job mentioned above describing the 
    capability of Praveen in fulfilling their needs. Also caution on giving context, mention only skills that
    known my praveen which relavent to the skills in {job_description}
    
    Remember you are Praveen. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [103]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res)})
print(res.content)

Subject: Application for the Machine Learning Engineer - Supply Chain position at Nike

Dear Hiring Manager,

I am writing to express my interest in the Machine Learning Engineer - Supply Chain position at Nike, as advertised. With my strong foundation in Python, SQL, and Amazon Web Services (AWS), along with my experience in machine learning, deep learning, and MLOps, I am confident in my ability to contribute to your AI/ML team and help deliver data-driven decisions that result in significant business impact.

Here's a brief overview of my skills and experience relevant to the job description:

1. Programming: Proficient in Python and SQL, with experience in developing end-to-end ML and MLOps pipelines.
2. Machine Learning & Deep Learning: Familiar with supervised and unsupervised learning techniques, neural networks, and large language models (LLMs).
3. Natural Language Processing (NLP): Experience in text processing, transformers, and language models.
4. Large Language Models (LLM)